In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!ls -al /kaggle/input

In [ ]:
!tree -d /kaggle/input

In [ ]:
!cat /kaggle/input/cat-dataset/CAT_03/00000811_011.jpg.cat

In [ ]:
!cat /kaggle/input/cat-dataset/cats/CAT_03/00000811_011.jpg.cat

In [ ]:
import glob
file_list = glob.glob('/kaggle/input/cat-dataset/CAT_03/*.jpg')

print(len(file_list))
print(file_list[:5])

In [ ]:
import matplotlib.pyplot as plt

import cv2
import numpy as np

cv_img = cv2.imread('/kaggle/input/cat-dataset/CAT_03/00000806_008.jpg')
cv_img = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
image = cv2.resize(cv_img, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)

print(type(image))
print(image.shape)
plt.imshow(image)
plt.show()

In [ ]:
!cat /kaggle/input/cat-dataset/CAT_03/00000806_008.jpg.cat

In [ ]:
import pandas as pd
file = '/kaggle/input/cat-dataset/CAT_03/00000806_008.jpg'
df = pd.read_csv(file+".cat", sep=" ", header=None)
df = df.drop(columns=[0, 19])
label = df.values.squeeze()

print(label.shape)
print(label)


# 디렉토리 내의 파일 읽기

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import cv2
import numpy as np
import pandas as pd


def load_image(file_name):
    cv_img = cv2.imread(file_name)
    return cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
#     return cv2.resize(cv_img, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)

def load_label(file_name):
    df = pd.read_csv(file_name+".cat", sep=" ", header=None)
    df = df.drop(columns=[0, 19])
    return df.values.squeeze()
    
file = '/kaggle/input/cat-dataset/CAT_03/00000806_008.jpg'
image_in_numpy = load_image(file)
label_in_numpy = load_label(file)
plt.imshow(image_in_numpy)
plt.show()
print(label_in_numpy)

In [ ]:
images = []
labels = []
for file_name in file_list:    
    images.append(load_image(file_name))
    labels.append(load_label(file_name))

# 이미지 리사이즈

In [ ]:
resized_images = []
resized_labels = []
for i in range(len(images)):
    resized_images.append(cv2.resize(images[i], dsize=(224, 224), interpolation=cv2.INTER_CUBIC))

    resized_label = np.empty_like(labels[i])
    resized_label[::2] = labels[i][::2] * 224 / images[i].shape[1]  # x, 가로
    resized_label[1::2] = labels[i][1::2] * 224 / images[i].shape[0]# y, 세로
    resized_labels.append(resized_label)

# Numpy 변환

In [ ]:
raw_x = np.array(resized_images, dtype=np.int)
raw_y = np.array(resized_labels, dtype=np.float)

print(raw_x.shape)
print(raw_y.shape)

# 딥러닝 학습

## 데이터 추리기

In [ ]:
x = raw_x
y = raw_y[:,0:4]

target_x = x[0]
target_y = y[0]

plt.imshow(target_x)
plt.scatter(target_y[0], target_y[1], color='yellow')
plt.scatter(target_y[2], target_y[3], color='yellow')
plt.show()

## Normalization

In [ ]:
x = x / 255.
y = y / 255.

## 데이터 분리

In [ ]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, shuffle=True)

print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

print(train_y[:10])

In [ ]:
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras import models
from tensorflow.keras import Model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.applications import mobilenet_v2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

conv_layers = mobilenet_v2.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
conv_layers.trainable = False

right_eye_model = models.Sequential()

right_eye_model.add(conv_layers)
right_eye_model.add(Flatten())
right_eye_model.add(Dense(128, activation='relu'))
right_eye_model.add(BatchNormalization())
right_eye_model.add(Dense(4, activation="sigmoid"))

right_eye_model.summary()

right_eye_model.compile(optimizer='Adam', loss='mse')


history = right_eye_model.fit(train_x, train_y, epochs=100, batch_size=16, validation_split=0.1, verbose=1)

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.show()


## 결과 보기

In [ ]:
predicted = right_eye_model.predict(test_x[:16])

for i in range(16):
  plt.imshow(test_x[i])
  plt.scatter(int(predicted[i][0]*255), int(predicted[i][1]*255), color='yellow')
  plt.scatter(int(predicted[i][2]*255), int(predicted[i][3]*255), color='yellow')
  plt.show()